# Find the cars available for sale 



In [ ]:
from flask import Flask, render_template, request, redirect
from datetime import datetime
import mysql.connector
from mysql.connector import connect, Error

In [ ]:
connection = connect(
    host = "localhost",
    user = "root",
    password = "***********",
    database = "vehiclesearch"
    )

cursor = connection.cursor()

In [ ]:
def getvehicles(cursor,zipcode,miles):
    
    stmt1 = "Select latitude, longitude from zipinfo where zipcode =" + str(zipcode)
    cursor.execute(stmt1)
    res1 = cursor.fetchall()
    
    # Coordinates of entered zipcode
    try:
        lat = res1[0][0]
        lon = res1[0][1]
    except:
        return "null"
    
    distance = "( 3959 * acos( cos( radians( " + str(lat) + ") ) * cos( radians( zf.latitude ) )* cos( radians( zf.longitude ) - radians("+ str(lon) + ") ) + sin( radians(" + str(lat) + ") ) * sin(radians(zf.latitude)) ) ) AS distance"
    
    stmt3 = """SELECT ls.year as year, ls.make as make, ls.model as model, ls.price as price, dlr.dealername as dealername,
            zf.city as city, zf.state as state, ls.zip as zipcode,""" + distance +"""
            From listings ls LEFT JOIN zipinfo zf on ls.zip = zf.zipcode 
            LEFT JOIN dealers dlr on ls.dealer_number = dlr.dealernumber"""
    
    final_stmt = """Select ft.year, ft.make, model, ft.price, ft.dealername, ft.city, ft.state, ft.zipcode FROM (""" + stmt3 + """) as ft
                WHERE ft.distance <="""+ str(miles) + """ ORDER BY distance"""
    
    cursor.execute(final_stmt)
    
    data = cursor.fetchall()
    
    return data

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def Search():
    
    if request.method=='POST':
        zipcode = request.form['zipcode']
        miles = float(request.form['miles'])
        
        data = getvehicles(cursor,zipcode,miles)
        
        if data == "null":
            return render_template('invalid.html')
        
        return render_template('index.html',data=data)
        
    return render_template('index.html')
    

In [ ]:
if __name__ == '__main__':
    app.run()